# Genome Annotation

Materials for a BRAKER & TSEBRA Genome Annotation workshop by Katharina Hoff (katharina.hoff@uni-greifswald.de).

Please find slides for an introductory talk on genome annotation (with BRAKER and TSEBRA) at [braker_erga_2022.pdf](braker_erga_2022.pdf)

In the following, we will walk through the process of genome annotation on the example of a small proportion of the *Arabidopsis thaliana* genome.

## Repeat masking

Repetitive sequences are a huge problem for genome annotation. Some repeats only coincidentally look like protein-coding genes, others (such as transposases) are protein-coding genes, but we usually are not interested in any of these "repeat genes" when trying to find protein-coding genes in a novel genome. Thus, a genome should be repeat-masked prior gene prediction. 

Repeat masking is a resource and time-consuming step that is out of scope for this workshop. We recommend using RepeatModeler2 (paper at https://doi.org/10.1073/pnas.1921046117 , software at https://www.repeatmasker.org/RepeatModeler/ ) to construct a species-specific repeat library and mask the genome with RepeatMasker (ideally, you will perform these computations on a node with >70 threads, in a place with very fast storage i/o, possibly using RAM instead of actual hard drive as a temporary file storage place):

```
T=72 # you need a large number of threads and fast i/o storage
GENOME=genome.fa
DB=some_db_name_that_fits_to_species

BuildDatabase -name ${DB} ${GENOME}
RepeatModeler -database ${DB} -pa ${T} -LTRStruct
RepeatMasker -pa 72 -lib ${DB}-families.fa -xsmall ${GENOME}
```

This results in a file `${GENOME}.masked`. 

<details>
  <summary>Click to learn how to mask more rigorously when needed</summary>
Depending on the kind of genome, plenty of unmasked repeats may still persist. This is generally an issue to be expected in large genomes, such as vertebrate genomes, and you will notice the problem if the count of predicted proteins is extremely high. You can try to overcome "under-masking" with the following steps (we are suggesting to use GNU parallel to speed up the process):

```
ln -s genome.masked.fa genome.fa
splitMfasta.pl --minsize=25000000 ${GENOME}.masked

# Running TRF
ls genome.split.*.fa | parallel 'trf {} 2 7 7 80 10 50 500 -d -m -h'

# Parsing TRF output
# The script parseTrfOutput.py is from https://github.com/gatech-genemark/BRAKER2-exp
ls genome.split.*.fa.2.7.7.80.10.50.500.dat | parallel 'parseTrfOutput.py {} --minCopies 1 --statistics {}.STATS > {}.raw.gff 2> {}.parsedLog'

# Sorting parsed output..."
ls genome.split.*.fa.2.7.7.80.10.50.500.dat.raw.gff | parallel 'sort -k1,1 -k4,4n -k5,5n {} > {}.sorted 2> {}.sortLog'

# Merging gff...
FILES=genome.split.*.fa.2.7.7.80.10.50.500.dat.raw.gff.sorted
for f in $FILES
do
    bedtools merge -i $f | awk 'BEGIN{OFS="\t"} {print $1,"trf","repeat",$2+1,$3,".",".",".","."}' > $f.merged.gff 2> $f.bedtools_merge.log
done

# Masking FASTA chunk
ls genome.split.*.fa | parallel 'bedtools maskfasta -fi {} -bed {}.2.7.7.80.10.50.500.dat.raw.gff.sorted.merged.gff -fo {}.combined.masked -soft &> {}.bedools_mask.log'

# Concatenate split genome
cat genome.split.*.fa.combined.masked > genome.fa.combined.masked
```

The file `genome.fa.combined.masked` will be more rigorously masked.
</details>

## RNA-Seq alignment with HiSat2

Spliced alignments of RNA-Seq short reads are a valuable information source for predicting protein-coding genes with high accuracy.

![Figure 3 of Lomsadze et al. (2014)](et-rnaseq.png "Figure 3 of Lomsadze et al. (2014) illustrates the use of RNA-Seq spliced alignments for predicting genes (with GeneMark-ET).")
Figure 3 of Lomsadze et al. (2014) illustrates the use of RNA-Seq spliced alignments for predicting genes (with GeneMark-ET), image source: https://doi.org/10.1093/nar/gku557.

We will map the *Arabidopsis thaliana* Illumina RNA-Seq reads from library SRR934391 in files [~/alphafold_data/response/sra/SRR934391_1.fastq.gz](~/alphafold_data/response/sra/SRR934391_1.fastq.gz) and [~/alphafold_data/response/sra/SRR934391_2.fastq.gz](~/alphafold_data/response/sra/SRR934391_2.fastq.gz). These are paired-end data, i.e. one file contains the forward reads while the other contains in the same order the reverse reads. The length of reads is in this case 100 nt.

We will use HiSat2 (publication at https://doi.org/10.1038/s41587-019-0201-4 , software at https://github.com/DaehwanKimLab/hisat2) to align these reads against a chunk of the *Arabidopsis thaliana* genome contained in the file [genome.fa](genome.fa). (You can in principle use any alignment tool capable of aligning RNA-seq reads to a genome, as long as it can perform spliced alignment.)

First, we need to build an index from the genome file:

In [1]:
%%script bash

# a toy data set to be used in this session is stored within the docker container
# if not done so, yet, we copy that file into our current working directory
if ! [ -f genome.fa ]
then
    ln -s /opt/BRAKER/example/genome.fa genome.fa
fi

# building the hisat2 index
hisat2-build genome.fa genome-idx 1> hisat2-build.log 2> hisat2-build.err

Inspect the log files [hisat2-build.log](hisat2-build.log) and [hisat2-build.err](hisat2-build.err) for possible errors.

Next, we align the RNA-seq reads against the genome. Consider to **not** do this during the session. Performing this alignment took about 7 minutes with 70 threads. The precomputed output file is provided at `${HOME}/alphafold_data/response/sra/SRR934391.sam`, and we will continue to use that pre-computed file.

In [20]:
%%script bash

T=6 # adjust to number of threads that you booted with

RNASEQDIR=${HOME}/alphafold_data/response/sra

time hisat2 -p ${T} -q -x genome-idx -1 ${RNASEQDIR}/SRR934391_1.fastq.gz \
    -2 ${RNASEQDIR}/SRR934391_2.fastq.gz -S rnaseq.sam \
    1> hisat2-align.log 2> hisat2-align.err


real	5m17.120s
user	15m4.275s
sys	11m22.905s


Our goal is to extract information on spliced alignments/intron positons from the alignment output file. To achieve this, we will use a tool called bam2hints that is part of the Augustus software suite (software at https://github.com/Gaius-Augustus/Augustus ). However, this tool requires a sorted bam-file. Therefore, we first use Samtools (paper at https://doi.org/10.1093/bioinformatics/btp352 , software at https://github.com/samtools ) to convert the sam file to bam format:

In [5]:
%%script bash

T=6 # adjust to number of threads that you booted with, takes ~2 minutes with 6 threads

SAMFILE=${HOME}/alphafold_data/response/sra/SRR934391.sam

samtools view -@${T} -bSh ${SAMFILE} -o rnaseq.bam

# if you computed your own rnaseq.sam file, delete it to save space on harddrive
if [ -f rnaseq.sam ]
then
    rm rnaseq.sam
fi

Then, we sort that bam file:

In [ ]:
%%script bash

T=6 # adjust to number of threads that you booted with, takes ~2 minutes with 6 threads

samtools sort -@${T} -n rnaseq.bam -o rnaseq.s.bam

# remove the unsorted bam file to save space
rm rnaseq.bam

## Annotation of protein coding genes

Structural genome annotation is ideally performed by a combination of a statistical model (e.g. Hidden Markov Model derivate) and extrinsic evidence (e.g. from transcriptomics). The statistical model parameters have to be adapted to the genomic properties of novel species. For adapting parameters, an initial set of high-quality training genes from the target species is required. This is tricky to obtain. BRAKER is a perl script that comprises several pipelines to automated the solution of this problem: fully automatically generate an initial set of training genes, train gene finders, and then predict genes with the trained parameters and extrinsic evidence.

We will take an approach to structural genome annotation that takes advantage both of RNA-Seq supported gene prediction with BRAKER1 (paper at https://doi.org/10.1093/bioinformatics/btv661, all BRAKER software at https://github.com/Gaius-Augustus/BRAKER) and protein supported gene prediction with BRAKER2 (paper at https://doi.org/10.1093/nargab/lqaa108). This will result in a total of four gene sets (1 from GeneMark-ET, 1 from GeneMark-EP, two from AUGUSTUS) that we will combine with TSEBRA (paper at https://doi.org/10.1186/s12859-021-04482-0, software at https://github.com/Gaius-Augustus/TSEBRA).

### Installation of GeneMark-ES/ET/EP

GeneMark-ES/ET/EP (publications at https://doi.org/10.1093/nar/gku557 , https://doi.org/10.1093/nargab/lqaa026 , https://doi.org/10.1101/gr.081612.108 ) is a dependency of BRAKER. It was not possible to compile this software into the Docker container due to a license issue. Therefore, everyone has to download and install the software by themselves. Perform the following steps:

   * Go to http://exon.gatech.edu/GeneMark/license_download.cgi
   * Select GeneMark-ES/ET/EP ver 4.69_lic LINUX 64 kernel 2.6 - 3
   * Fill in Name = "Your name", Institution = "University of Greifswald", Country = "Germany"
   * Click on agree to the license agreement

On the next website, click on "here" to download **gmes_linux_64.tar.gz**, and click on "64_bit" to download a file **gm_key_64.gz**.

In the AppHub, upload both files into your home directory.

Execute the following code (**after downloading both files and uploading them into the AppHub**) to install both GeneMark-ES/ET/EP and ProtHint, a dependency of GeneMark-EP:

In [ ]:
%%script bash

# if you already have a bashrc file, load environment from there
if [ -f ${HOME}/.bashrc ]; then
    source ${HOME}/.bashrc
fi

if [ ! -d ${HOME}/gmes_linux_64 ]  && [ -f ${HOME}/gmes_linux_64.tar.gz ] && [ -f ${HOME}/gm_key_64.gz ]
# if no extracted gmes_linux_64 folder exists yet and if you uploaded both files
then
    # go to your home directory
    cd ${HOME}
    # unpack the uploaded genemark software archive
    gunzip gmes_linux_64.tar.gz
    tar -xf gmes_linux_64.tar
    # expand path variable to find binaries
    export PATH=${PATH}:${HOME}/gmes_linux_64
    # writh path expansion to bashrc file
    echo "export PATH=${PATH}" >> ${HOME}/.bashrc
    # change to genemark directory
    cd gmes_linux_64
    # change perl binary in all scripts according to Docker container
    change_path_in_perl_scripts.pl "/opt/conda/bin/perl" &> /dev/null
    # change to home directory
    cd ${HOME}
    # unpack the license key (will expire after 200 days)
    gunzip gm_key_64.gz
    # place key in correct location
    mv gm_key_64 ${HOME}/.gm_key
    # go to home directory
    cd ${HOME}
    # download ProtHint pipeline (dependency of BRAKER)
    git clone https://github.com/gatech-genemark/ProtHint.git &> /dev/null
    # copy GeneMark-ES into ProtHint directory
    cp -r ${HOME}/gmes_linux_64/* ProtHint/dependencies/GeneMarkES
    # expand path to find binaries
    export PATH=${PATH}:${HOME}/ProtHint/bin
    # write path expansion to bashrc file
    echo "export PATH=${PATH}" >> ${HOME}/.bashrc
else
    echo "Did not install GeneMark-ES/ET/EP."
    echo "Either it was already installed (two executable files can be found below), or you need to ask for help with the installation"
fi

which gmes_petap.pl # representative for GeneMark-ES/ET/EP
which prothint.py # representative for the other scripts

### BRAKER1

BRAKER1 uses spliced alignment information from RNA-Seq for training GeneMark-ET, for selecting a training gene set for AUGUSTUS, and for predicting genes with AUGUSTUS. For this, the bam-file with RNA-Seq information needs to be converted to "hints". BRAKER can perform this step automatically, but it can take a long time. In particular if a lot of RNA-Seq libraries and a limited job runtim on a HPC system, you should perform this step prior running BRAKER for all the libraries in parallel. Therefore, we do it separately in this session. If you want to skip this step, the output file is provided at [rnaseq.hints](rnaseq.hints):

In [ ]:
%%script bash

bam2hints --intronsonly --in=rnaseq.s.bam --out=rnaseq.hints # takes 3-4 minutes

BRAKER will execute fully automated training of the gene finders GeneMark-ET and AUGUSTUS. For AUGUSTUS, a configuration directory where you have writing permissions is required. Therefore, we will in the next cell create the required data structure (this has to be done only once, even if you run BRAKER many more times, later; note that this step will become obsolete when we release BRAKER3 in January 2023):

In [1]:
%%script bash

if ! [ -d config ]
then
    cp -r /usr/share/augustus/config .
fi

if [ -f rnaseq.s.bam ]; then
    rm rnaseq.s.bam # also delete that bam file because you previously generated hints from it
fi

Next, we will run BRAKER to predict genes in the genomic sequence with the prepared RNA-Seq intron evidence. Training AUGUSTUS for a novel species usually comprises a step called `etraining` that adapts species-specific parameters of the statistical model of AUGUSTUS, and a step called `optimize_augustus.pl` that optimizes meta-parameters of that model. `optimize_augustus.pl` is very time-consuming. For this session, will disable this step with `--skipOptimize`. If you ever want to annotate a real new genome, make sure to delete `--skipOptimize` from your BRAKER calls (and expect substantially longer runtime):

In [ ]:
%%script bash

T=6 # adjust to number of threads that you booted with, takes ~2.5 minutes on 6 threads

source ${HOME}/.bashrc # load GeneMark location

export AUGUSTUS_CONFIG_PATH="${PWD}/config" # tell BRAKER where to find writable AUGUSTUS parameters

# delete output from a possible previous run if it exists
if [ -d BRAKER1 ]
then
    rm -rf BRAKER1
fi

braker.pl --workingdir=BRAKER1 --genome=genome.fa --hints=rnaseq.hints --softmasking \
    --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ \
    --cores ${T} --gm_max_intergenic 10000 \
    --skipOptimize #  remember to remove this option if you are running a real job
    # this call takes a few minutes even with --skipOptimize

Note that BRAKER by default expects scripts and binaries in a location relative to the `$AUGUSTUS_CONFIG_PATH`. We here changed the location of the `$AUGUSTUS_CONFIG_PATH` to a writable location. Therefore, we have to tell BRAKER where the scripts and binaries are (`--AUGUSTUS_BIN_PATH`, `--AUGUSTUS_CONFIG_PATH`).

The most important output files that we will later use for running TSEBRA are 

   * [BRAKER1/augustus.hints.gtf](BRAKER1/augustus.hints.gtf)
   * [BRAKER1/GeneMark-ET/genemark.gtf](BRAKER1/GeneMark-ET/genemark.gtf)
   * [BRAKER1/hintsfile.gff](BRAKER1/hintsfile.gff)
   
The file [BRAKER1/what-to-cite.txt](BRAKER1/what-to-cite.txt) advises you on what papers should be cited if you were going to publish a manuscript on a gene set produced with BRAKER1.
   
### BRAKER2

BRAKER2 uses spliced alignment information from a huge database of proteins against the target genome. Note: a set of proteins from one or a few related species is not sufficient for running BRAKER2 (in such a situation, consider using GALBA, instead, software at https://github.com/Gaius-Augustus/GALBA). A particular set of proteins of a closely related species can be appended to a larger database for running BRAKER2. However, BRAKER2 is not an ideal tool for recovering a complete set of proteins from a related species.

The following call of BRAKER takes ~9 minutes on 6 threads, even when optimizing AUGUSTUS parameters is disabled. Consider **not** executing this during the live session, precomputed results are made available.

In [ ]:
%%script bash

T=6 # adjust to number of threads that you booted with

ORTHODB=${HOME}/alphafold_data/response/prothint_db/odb10_plants.fasta # adjust to suitable clade

source ${HOME}/.bashrc # load GeneMark location

export AUGUSTUS_CONFIG_PATH=${PWD}/config

# delete output from a possible previous run if it exists
if [ -d BRAKER2 ]
then
    rm -rf BRAKER2
fi

time braker.pl --workingdir=BRAKER2 --genome=genome.fa \
    --prot_seq=${ORTHODB} --softmasking \
    --AUGUSTUS_BIN_PATH=/usr/bin/ \
    --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ \
    --cores ${T} --gm_max_intergenic 10000 \
    --skipOptimize \ # remember to remove this option if you are running a real job
    2> braker_prothint.log

The most important output files that we will later use for running TSEBRA are 

   * [BRAKER2/augustus.hints.gtf](BRAKER2/augustus.hints.gtf)
   * [BRAKER2/GeneMark-EP/genemark.gtf](BRAKER2/GeneMark-EP/genemark.gtf)
   * [BRAKER2/hintsfile.gff](BRAKER2/hintsfile.gff)
   
The file [BRAKER2/what-to-cite.txt](BRAKER2/what-to-cite.txt) advises you on what papers should be cited if you were going to publish a manuscript on a gene set produced with BRAKER2.

### TSEBRA

TSEBRA is a tool for selecting a highly accurate gene set from several input sets according to supporting extrinsic evidence. We provide some background in [TSEBRA_Workshop.pdf](TSEBRA_Workshop.pdf). Here is how we run TSEBRA, today:

In [23]:
%%script bash

tsebra.py -g BRAKER1/augustus.hints.gtf,BRAKER1/GeneMark-ET/genemark.gtf,BRAKER2/augustus.hints.gtf,BRAKER2/GeneMark-EP/genemark.gtf \
    -e BRAKER1/hintsfile.gff,BRAKER2/hintsfile.gff -o tsebra.gtf 2> tsebra.log

Check the file [tsebra.log](tsebra.log) for possible errors. The final gene set is in file [tsebra.gtf](tsebra.gtf).

## BUSCO assessment

BUSCO (paper at https://doi.org/10.1002/cpz1.323, software at https://gitlab.com/ezlab/busco) can provide information on sensitivity with respect to a clade-specific core gene set. We will in the following extract the amino acid sequences of predicted genes, and obtain BUSCO scores for all gene sets that went into TSEBRA, and for the final gene set.

In [24]:
%%script bash

# getAnnoFastaFromJoingenes.py does not have executability permission in Docker container
# therefore make softlink for calling it with python explicitely
if ! [ -f getAnnoFastaFromJoingenes.py ]
then
    ln -s /usr/share/augustus/scripts/getAnnoFastaFromJoingenes.py getAnnoFastaFromJoingenes.py
fi

python getAnnoFastaFromJoingenes.py -g genome.fa \
    -f BRAKER1/augustus.hints.gtf -o b1-augustus

python getAnnoFastaFromJoingenes.py -g genome.fa \
    -f BRAKER1/GeneMark-ET/genemark.gtf -o genemark-et

python getAnnoFastaFromJoingenes.py -g genome.fa \
    -f BRAKER2/augustus.hints.gtf -o b2-augustus

python getAnnoFastaFromJoingenes.py -g genome.fa \
    -f BRAKER2/GeneMark-EP/genemark.gtf -o genemark-ep

python getAnnoFastaFromJoingenes.py -g genome.fa \
    -f tsebra.gtf -o tsebra

This generated the following files with protein sequences:

   * [b1-augustus.aa](b1-augustus.aa)
   * [genemark-et.aa](genemark-et.aa)
   * [b2-augustus.aa](b2-augustus.aa)
   * [genemark-ep.aa](genemark-ep.aa)
   * [tsebra.aa](tsebra.aa)
   
Let's have a quick look at the number of transcript products in each file ($\neq$ number of genes):

In [25]:
%%script bash
grep -c ">" *.aa

b1-augustus.aa:278
b2-augustus.aa:300
genemark-ep.aa:279
genemark-et.aa:266
tsebra.aa:318


First, we find the closest BUSCO lineage (we are working on *Arabidopsis thaliana*):

In [27]:
%%script bash

source conda_init
conda activate busco_env

busco --list-datasets > busco_lineages.txt 2> busco_lineages.log

All available lineages are now in [busco_lineages.txt](busco_lineages.txt). (Check [busco_lineages.log](busco_lineages.log) for possible errors.)

Check at NCBI taxonomy (https://www.ncbi.nlm.nih.gov/taxonomy) the lineage of the target *Arabidopsis*. I believe the lineage is:

`cellular organisms; Eukaryota; Viridiplantae; Streptophyta; Streptophytina; Embryophyta; Tracheophyta; Euphyllophyta; Spermatophyta; Magnoliopsida; Mesangiospermae; eudicotyledons; Gunneridae; Pentapetalae; rosids; malvids; Brassicales; Brassicaceae; Camelineae`

Now find a related lineage in [busco_lineages.txt](busco_lineages.txt). `brassicales_odb10` is the closest lineage.

Next, we run a BUSCO assessment on all gene sets (this takes ~4 minutes with 6 threads):

In [28]:
%%script bash

source conda_init
conda activate busco_env

T=6 # adjust to number of threads that you booted with

GENESETS=(b1-augustus genemark-et b2-augustus genemark-ep tsebra)

for s in ${GENESETS[@]}; do
    echo "Processing ${s}..."
    if [ -d busco_${s} ]
    then
        rm -r busco_${s}
    fi
    busco -m proteins -i b1-augustus.aa -o busco_${s} \
        -l brassicales_odb10 -c ${T} &> busco_${s}.log
done

Processing b1-augustus...
Processing genemark-et...
Processing b2-augustus...
Processing genemark-ep...
Processing tsebra...


Next, we visualize the BUSCO results:

In [29]:
%%script bash

source conda_init
conda activate busco_env

if ! [ -d BUSCO_summaries ]
then
    mkdir BUSCO_summaries
fi

cp busco_*/short_summary*.txt BUSCO_summaries

generate_plot.py -wd BUSCO_summaries &> generate_plot.log

Check the file [generate_plot.log](generate_plot.log) for possible errors. This results in the following figure (stored at [BUSCO_summaries/busco_figure.png](BUSCO_summaries/busco_figure.png)):

<img src="BUSCO_summaries/busco_figure.png" alt="BUSCO results" width="1000"/>

The data that we used in this session was selected purely on the criterion of feasible runtime. In a real scenario, with a complete genome, the BUSCO plot should look more like this (sensitivity should increase in the final TSEBRA set):

<img src="busco_ideally.png" alt="BUSCO results (ideally)" width="1000"/>

## Data visualization in the UCSC Genome Browser

Visualization of gene structures in context with extrinsic evidence is essential for coming to a decision on whether a gene set "makes sense" or "does not make sense". Typical problems that you may observe in a genome browser include "split genes" (where evidence implies two genes should in fact be a single gene) or "joined genes" (where evidence implies one gene should be split into two genes).

The UCSC Genome Browser (publication at https://doi.org/10.1101/gr.229102) is one of the most popular genome browsers. It has the advantage that you do not have to install a browser instance on your own webserver. Instead, you only need to provide a certain data structure with your target data on a webserver. The UCSC Genome Browser servers can display your data from there. The data structures are called "track data hubs" or "assembly hubs" (paper at https://doi.org/10.1093/bioinformatics/btt637). 

MakeHub (paper at https://doi.org/10.1016/j.gpb.2019.05.003 , software at https://github.com/Gaius-Augustus/MakeHub ) is a python script that fully automates the generation of such track data hubs for novel genomes. In the following, we will generate a simple track data hub for the genome sequence that we annotated with BRAKER and TSEBRA (before starting this job, check whether you have around 10G of RAM available, top right corner of your web browser window):

In [ ]:
%%script bash

T=6 # adjust to number of threads that you booted with

# merge hints
cat BRAKER1/hintsfile.gff BRAKER2/hintsfile.gff > all_hints.gff

make_hub.py -e katharina.hoff@uni-greifswald.de \
    --genome genome.fa --long_label "A chunk from the Arabidopsis thaliana genome" \
    --short_label at_chunk  --bam rnaseq.s.bam --cores ${T} \
    --display_bam_as_bam --latin_name "Arabidopsis thaliana" \
    --assembly_version "artifically split custom assembly" \
    --hints all_hints.gff --gene_track tsebra.gtf TSEBRA

You can't perform the suggested `scp` command from the apphub, unless you have privileges on a University of Greifswald webserver. We have therefore copied a prepared hub in advance. The `hub.txt` is available at https://bioinf.uni-greifswald.de/hubs/at_chunk/hub.txt . Remember that link.

In order to visualize your data, go to https://genome.ucsc.edu/ . Click on `My Data` -> `Track Hubs` -> choose the European mirror -> click on `Connected Hubs` and enter the link https://bioinf.uni-greifswald.de/hubs/at_chunk/hub.txt into the text window -> click on `Add Hub`. Congratulations, your Hub is now connected. You should be able to browse something like this: 

<img src="at_chunk.png" alt="UCSC Genome Browser example" width="1000"/>

### How to know which sequences to browse

The long sequences are usually the most interesting to look at. The following command gives you the names of sequences in the order of descending length, you can copy-paste the sequence names into the search window in the UCSC Genome Browser.

In [30]:
%%script bash

N=5 # how many longest sequences would you like to know about

summarizeACGTcontent.pl genome.fa | grep bases | head -${N} | sort -n \
   | perl -ne 'm/(\d+)\s+bases\.\s+(\S+)/; print "$2\t$1\n";'

X1	125040
X2	125100
X3	125100
X4	125100
X5	125100


## How to run BRAKER (and other software) in Docker

If you have a machine on which you have root permissions and Docker, you can run the exact same container as we have been using during this workshop as follows:

```
sudo docker run --rm -it -u root katharinahoff/response-notebook:devel bash
```

You can execute all shell commands that we covered in this notebook in that container. Note: you will have to re-copy the `$AUGUSTUS_CONFIG_PATH` and re-install GeneMark-ES/ET/EP. You might find the following tutorial helpful: https://www.tutorialspoint.com/how-to-copy-files-from-host-to-docker-container#:~:text=Another%20way%20to%20copy%20files,folder%20in%20the%20host%20machine.

## How to run BRAKER (and other software) in Singularity

If your HPC/server has Singularity, you I recommend to use a different docker image. It is a prerequisite that before executing that container, you download and install GeneMark-ES/ET/EP and the key file in your host home directory, and that you remember to which location that software was installed.

The "BRAKER" container is not containing exactly the same version of BRAKER as the Docker container. Instead, it contains the BRAKER1 and BRAKER2 versions that will be included in BRAKER3. Results of the two containers might therefore differ slightly. The BRAKER container has been adapted to the needs of HPC SLURM Singularity users.

```
if ! command -v singularity; then
    module load singularity
fi

if ! command -v singularity; then
    echo "Please install singularity!"
fi

singularity build braker.sif docker://katharinahoff/braker-notebook:devel
```

Here is an example call for BRAKER1:

```
singularity exec -B $PWD:$PWD braker.sif braker.pl --genome=genome.fa --hints=rnaseq_hints.gff --softmasking --threads=24 --gm_max_intergenic 10000 --GENEMARK_PATH=${HOME}/bin/gm_et_linux_64/gmes_petap --workingdir=braker1
```
You have to adapt the location of gmes_petap to your system, and you have to adapt input and output names.

Here is an example call for BRAKER2:

```
singularity exec -B $PWD:$PWD braker.sif braker.pl --genome=genome.fa --prot_seq=orthodb.fa --softmasking --threads=24 --gm_max_intergenic 10000 --GENEMARK_PATH=${HOME}/bin/gm_et_linux_64/gmes_petap --workingdir=braker2 
```

This new version of BRAKER does not produce output files augustus.hints.gtf, anymore. Instead, it writes output files called braker.gtf (which contains the combined gene set of AUGUSTUS and GeneMark-ES/ET/EP). Subsequently, the TSEBRA call is much simpler:

```
singularity exec -B $PWD:$PWD braker.sif tsebra.py -g braker1/braker.gtf,braker2/braker.gtf \
    -e braker1/hintsfile.gff,braker2/hintsfile.gff -o tsebra.gtf
```

The BRAKER container does not contain BUSCO, but it contains Hisat2, Samtools, and MakeHub. They can be used in a similar way as described for BRAKER1, BRAKER2, and TSEBRA.

## Troubleshooting

### I have 80.000 genes predicted by BRAKER/TSEBRA in a full genome, what shall I do?

Please first check whether you are referring to genes, or to transcripts. BRAKER predicts alternative isoforms. If RNA-Seq data supports this, the number of alternative transcripts may be large, but likely true. If it's really genes that you counted, then 80.000 sounds way too much, indeed. Most likely, GeneMark-ET/ES/EP produced highly fragmented training genes for AUGUSTUS. This will also lead to highly fragmented genes predicted by AUGUSTUS. First, check whether you genome has been masked for repeats. Consider using the additional TRF masking desribed at the top of this notebook. If that does not help, and if you have a protein set of closely related species at hand, consider using that protein set as sole training data for AUGUSTUS. You can use GALBA for this (https://github.com/Gaius-Augustus/GALBA). Also, keep an eye out for the appearance of BRAKER3 because that may also fix your problem.

### I have only 10.000 genes predicted by BRAKER/TSEBRA in a full genome, what shall I do?

Check whether the BRAKER output files produced more genes than TSEBRA. By default, TSEBRA will discard genes without evidence. If you have only little evidence for your species, TSEBRA might be a bad idea.

### How do I know how many genes to expect?

Hard to say. You can download gene sets of related species e.g. from NCBI Genomes, and count. Some gene sets tend to be "underannotated", i.e. they may represent rather the lower numbers of what might be realistic. Katharina usually gets nervous about more than 40000 genes and fewer than 20000 genes. These are definitely weird gene counts. Otherwise: always inspect your gene set in a Genome Browser such as the UCSC Genome Browser to identify problems.

### I have long isoseq RNA-Seq transcripts, can I put them into BRAKER?

No. But we have other instructions for you at https://github.com/Gaius-Augustus/BRAKER/blob/master/docs/long_reads/long_read_protocol.md . Please note: isoseq data does not always aid structural genome annotation over short read RNA-Seq data.

### The BUSCO scores of my genome are higher than those of my protein, why?

Some short housekeeping genes are commonly masked during repeat masking. They won't show up in a BRAKER gene set. You can manually add them from BUSCO output to a final gene set. We currently do not provide an automated workflow for this.

### I opened an issue on GitHub about BRAKER or TSEBRA 100 days ago, nobody replied, why?

We are a small team of developers. We try our best and usually respond to well described and easy-to-solve issues within a rather short time frame. Solving other issues may take considerable amounts of time that we simply do not have, or they may be described in a way that we don't know what do with them... please be patient with us.

### I have a problem, whom do I tell?

Please read through the Issues on Github. If the issue does not exist, yet, open an issue.

### The End